# Legal Text Annotation and Named-Entity Recognition on Power of Attorney Documents

- Shubhangi Shrivastava
- Utsav Patel

In [1]:
!pip install flair

     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 798kB 15.2MB/s 
     |████████████████████████████████| 133kB 29.0MB/s 
     |████████████████████████████████| 1.0MB 36.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/70/55/6b/c12cf58209b8346f653a04f37dd8f607ab0e85a26238a23420
  Stored in directory: /root/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
Successfully built flair segtok mpld3 sqlitedict


In [0]:
import flair
import numpy as np

from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from typing import List

from flair.models import SequenceTagger

from flair.trainers import ModelTrainer

from flair.visual.training_curves import Plotter

In [0]:
columns = {0: 'text', 1: 'pos', 2: 'ner'}

In [0]:
data_folder = './'

In [6]:
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                              train_file='train.txt')
#   ,
#                                                               test_file='test.txt',
#                                                               dev_file='dev.txt')
print(corpus)

2019-05-04 00:22:35,444 Reading data from .
2019-05-04 00:22:35,446 Train: train.txt
2019-05-04 00:22:35,447 Dev: None
2019-05-04 00:22:35,448 Test: None
2019-05-04 00:22:35,452 UTF-8 can't read: train.txt ... using "latin-1" instead.
TaggedCorpus: 222 train + 25 dev + 28 test sentences


In [0]:
tag_type = 'ner'

In [8]:
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'PER', b'COMP', b'DATE', b'ORG', b'WITNESS', b'PER|*', b'*', b'DATE|DATE', b'WITNESS\\_1', b'<START>', b'<STOP>']


In [9]:
embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
#      FlairEmbeddings('news-forward'),
#      FlairEmbeddings('news-backward'),
]

2019-05-04 00:22:39,809 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpgemupv6i


100%|██████████| 160000128/160000128 [00:10<00:00, 14825006.58B/s]

2019-05-04 00:22:51,340 copying /tmp/tmpgemupv6i to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2019-05-04 00:22:51,855 removing temp file /tmp/tmpgemupv6i
2019-05-04 00:22:52,480 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpazf1mhl6


100%|██████████| 21494764/21494764 [00:02<00:00, 8837666.72B/s]

2019-05-04 00:22:55,625 copying /tmp/tmpazf1mhl6 to cache at /root/.flair/embeddings/glove.gensim
2019-05-04 00:22:55,658 removing temp file /tmp/tmpazf1mhl6
2019-05-04 00:22:55,660 this function is deprecated, use smart_open.open instead


In [0]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [0]:
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_rnn=True)

In [0]:
trainer : ModelTrainer = ModelTrainer(tagger, corpus)

In [0]:
trainer.train('./project/example/ner', learning_rate=0.3, mini_batch_size=32, max_epochs=100)

In [0]:
plotter = Plotter()
plotter.plot_training_curves('./project/example/ner/loss.tsv')
plotter.plot_weights('./project/example/ner/weights.txt')

**References**

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md